In [32]:
import pandas as pd
import numpy as np
import glob
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print('✓ Libraries loaded')

✓ Libraries loaded


## 1. Load và consolidate dữ liệu từ tất cả CSV files

In [33]:
# Cấu hình
CSV_DIR = 'dest_trends_raw'
ANCHOR = 'Rau má'
START_DATE = 'thg 1 2011'  # Tháng 1 năm 2011

# Tìm tất cả file CSV
csv_files = sorted(glob.glob(os.path.join(CSV_DIR, 'dest_group_*.csv')))
print(f"Found {len(csv_files)} CSV files")

# Load và NORMALIZE từng file trước khi merge
normalized_dfs = []
anchor_df = None  # Lưu anchor values từ file đầu tiên

for i, csv_file in enumerate(csv_files):
    try:
        df = pd.read_csv(csv_file)
        
        # Lưu anchor values từ file đầu tiên
        if anchor_df is None and ANCHOR in df.columns:
            anchor_df = df[['date', ANCHOR]].copy()
            print(f"Using anchor from {csv_file}")
        
        # Normalize ngay trong file này
        if ANCHOR in df.columns:
            anchor_values = df[ANCHOR].replace(0, np.nan)
            
            # Normalize tất cả columns (trừ date và anchor)
            df_normalized = df[['date']].copy()
            for col in df.columns:
                if col not in ['date', ANCHOR]:
                    df_normalized[col] = (df[col] / anchor_values) * 100
            
            normalized_dfs.append(df_normalized)
        
        if (i + 1) % 50 == 0:
            print(f"Processed {i + 1}/{len(csv_files)} files")
            
    except Exception as e:
        print(f"Error loading {csv_file}: {e}")

print(f"\nSuccessfully processed {len(normalized_dfs)} files")
print(f"Anchor values preserved from first file")

Found 242 CSV files
Using anchor from dest_trends_raw/dest_group_001.csv
Processed 50/242 files
Processed 100/242 files
Processed 150/242 files
Processed 100/242 files
Processed 150/242 files
Processed 200/242 files

Successfully processed 242 files
Anchor values preserved from first file
Processed 200/242 files

Successfully processed 242 files
Anchor values preserved from first file


## 2. Merge tất cả dữ liệu vào một DataFrame

In [34]:
# Merge tất cả normalized dataframes
# Sử dụng concat trên columns với outer join

prepared_dfs = []
for df in normalized_dfs:
    df_indexed = df.set_index('date')
    prepared_dfs.append(df_indexed)

# Concatenate
combined_df = pd.concat(prepared_dfs, axis=1, join='outer')

# Reset index
combined_df = combined_df.reset_index()

# Xóa duplicate columns
combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]

# Thêm anchor column từ anchor_df
if anchor_df is not None:
    combined_df = pd.merge(combined_df, anchor_df, on='date', how='left')
    # Đặt Rau má = 100 cho tất cả rows (vì nó là anchor)
    combined_df[ANCHOR] = 100.0

print(f"✓ Merge completed")
print(f"Combined DataFrame shape: {combined_df.shape}")
print(f"Total columns: {len(combined_df.columns)}")
print(f"Total rows (months): {len(combined_df)}")

✓ Merge completed
Combined DataFrame shape: (264, 969)
Total columns: 969
Total rows (months): 264


## 3. Cắt dữ liệu từ 1/1/2011 trở đi

In [35]:
# Filter dữ liệu từ 2011 trở đi (bằng cách extract năm từ date string)
def extract_year(date_str):
    """Extract year from Vietnamese date format like 'thg 1 2011'"""
    try:
        return int(date_str.split()[-1])
    except:
        return 0

combined_df['year'] = combined_df['date'].apply(extract_year)

# Filter từ 2011 trở đi
combined_df = combined_df[combined_df['year'] >= 2011].copy()

# Drop cột year tạm
combined_df = combined_df.drop('year', axis=1)

# Sort theo date để đảm bảo thứ tự đúng
combined_df = combined_df.sort_values('date').reset_index(drop=True)

print(f"✓ Filtered data from 2011 onwards")
print(f"New shape: {combined_df.shape}")
print(f"Date range: {combined_df['date'].iloc[0]} to {combined_df['date'].iloc[-1]}")
print(f"Total months: {len(combined_df)}")

# Lưu vào normalized_df để giữ tên biến nhất quán
normalized_df = combined_df.copy()

✓ Filtered data from 2011 onwards
New shape: (180, 969)
Date range: thg 1 2011 to thg 9 2025
Total months: 180


## 4. Normalize traffic theo anchor (Rau má)

In [36]:
# Dữ liệu đã được normalize ở bước trước
# Cell này chỉ để kiểm tra và xử lý NaN/Inf

print(f"✓ Data already normalized in previous step")
print(f"Total destinations: {len(normalized_df.columns) - 1}")
print(f"Anchor column '{ANCHOR}' = 100 (reference)")

# Xử lý NaN/Inf nếu có
normalized_df = normalized_df.replace([np.inf, -np.inf], np.nan)
normalized_df = normalized_df.fillna(0)

print(f"\n✓ Cleaned NaN/Inf values")

✓ Data already normalized in previous step
Total destinations: 968
Anchor column 'Rau má' = 100 (reference)

✓ Cleaned NaN/Inf values


## 5. Thống kê tổng quát

In [37]:
# Thống kê cho mỗi destination
stats_list = []

for col in normalized_df.columns:
    if col != 'date':
        non_zero = normalized_df[col][normalized_df[col] > 0]
        stats_list.append({
            'Destination': col,
            'Mean': normalized_df[col].mean(),
            'Median': normalized_df[col].median(),
            'Max': normalized_df[col].max(),
            'Min': normalized_df[col].min(),
            'Std Dev': normalized_df[col].std(),
            'Non-zero Count': len(non_zero),
            'Coverage %': (len(non_zero) / len(normalized_df)) * 100
        })

stats_df = pd.DataFrame(stats_list)

print(f"Statistics for all {len(stats_df)} destinations:")
print(f"\nTop 10 by Average Interest:")
print(stats_df.nlargest(10, 'Mean')[['Destination', 'Mean', 'Median', 'Max', 'Coverage %']])

print(f"\nOverall Summary:")
print(f"Total time periods: {len(normalized_df)}")
print(f"Total destinations: {len(stats_df)}")
print(f"Average coverage: {stats_df['Coverage %'].mean():.2f}%")

Statistics for all 968 destinations:

Top 10 by Average Interest:
          Destination         Mean      Median          Max  Coverage %
897          Lam Kinh  1067.921986  163.333333  8500.000000  100.000000
849           Bình An   807.656986  750.000000  1700.000000  100.000000
730          Phú Quốc   643.394484  664.583333  1300.000000   98.888889
100            Hồ Tây   400.804520  324.500000  1200.000000  100.000000
375        Đồng Thông   334.366562  295.553360  1000.000000  100.000000
647  Thành phố Hà Nội   286.560800  163.392857  1733.333333  100.000000
777      Phượng Hoàng   176.851879  150.000000   833.333333  100.000000
628             Ba Vì   175.048899  159.411765   500.000000  100.000000
941           Tam Đảo   151.717730  142.857143   375.000000  100.000000
869            Mũi Né   143.361527  136.363636   550.000000  100.000000

Overall Summary:
Total time periods: 180
Total destinations: 968
Average coverage: 27.25%


## 6. Preview dữ liệu

In [38]:
# Hiển thị một vài rows đầu
print("First 5 rows of normalized data:")
print(normalized_df.iloc[:5, :10])  # Hiển thị 10 cột đầu

print(f"\n...and {len(normalized_df.columns) - 10} more columns")

# Hiển thị thông tin về data types
print(f"\nData types:")
print(normalized_df.dtypes.value_counts())

First 5 rows of normalized data:
         date  Chợ Trung tâm Ba Chẽ  Miếu Ông – Miếu Bà  Phố đi bộ Tiên Yên  \
0  thg 1 2011                   0.0                 0.0                 0.0   
1  thg 1 2012                   0.0                 0.0                 0.0   
2  thg 1 2013                   0.0                 0.0                 0.0   
3  thg 1 2014                   0.0                 0.0                 0.0   
4  thg 1 2015                   0.0                 0.0                 0.0   

   căn cứ địa cách mạng Hải Chi  Văn hóa, Thể thao các dân tộc vùng Đông Bắc  \
0                           0.0                                          0.0   
1                           0.0                                          0.0   
2                           0.0                                          0.0   
3                           0.0                                          0.0   
4                           0.0                                          0.0   

   Đền thờ 

## 7. Lưu dữ liệu đã consolidate

In [39]:
# Lưu consolidated data
output_file = 'complete_destinations_normalized.csv'
normalized_df.to_csv(output_file, index=False)
print(f"✓ Saved consolidated data to '{output_file}'")
print(f"File size: {os.path.getsize(output_file) / 1024 / 1024:.2f} MB")

# Lưu statistics
stats_file = 'destinations_statistics.csv'
stats_df.to_csv(stats_file, index=False)
print(f"✓ Saved statistics to '{stats_file}'")

✓ Saved consolidated data to 'complete_destinations_normalized.csv'
File size: 1.22 MB
✓ Saved statistics to 'destinations_statistics.csv'
